In [5]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
from pprint import pprint
import pymongo

load_dotenv()

connector = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average, maximum and minimum number of trackpoints per user

##### Average

##### Maximum

##### Minimum

### Task 3

Find the top 15 users with the highest number of activities.

In [6]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id": {"user_id": "$user_id"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 15},
    ]
)

for row in rows:
    pprint(row)

{'_id': {'user_id': '025'}, 'count': 715}
{'_id': {'user_id': '128'}, 'count': 519}
{'_id': {'user_id': '062'}, 'count': 406}
{'_id': {'user_id': '041'}, 'count': 399}
{'_id': {'user_id': '004'}, 'count': 346}
{'_id': {'user_id': '140'}, 'count': 345}
{'_id': {'user_id': '017'}, 'count': 265}
{'_id': {'user_id': '003'}, 'count': 261}
{'_id': {'user_id': '014'}, 'count': 236}
{'_id': {'user_id': '030'}, 'count': 210}
{'_id': {'user_id': '011'}, 'count': 201}
{'_id': {'user_id': '039'}, 'count': 198}
{'_id': {'user_id': '034'}, 'count': 180}
{'_id': {'user_id': '000'}, 'count': 155}
{'_id': {'user_id': '002'}, 'count': 146}


### Task 4
Find all users who have taken a bus.

### Task 5
List the top 10 users by their amount of different transportation modes.

### Task 6
Find activities that are registered multiple times. You should find the query even
if it gives zero result.

Assumption: Registered multiple times, meaning duplicates with the same user_id, transportation_mode, start_date_time, end_date_time.

In [3]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {
            "$lookup": {
                "from": "Activity",
                "let": {
                    "a_id": "$_id",
                    "a_user_id": "$user_id",
                    "a_transportation_mode": "$transportation_mode",
                    "a_start_date_time": "$start_date_time",
                    "a_end_date_time": "$end_date_time",
                },
                "pipeline": [
                    {
                        "$match": {
                            "$expr": {
                                "$and": [
                                    {"$ne": ["$$a_id", "$_id"]},
                                    {"$eq": ["$$a_user_id", "$user_id"]},
                                    {
                                        "$eq": [
                                            "$$a_transportation_mode",
                                            "$transportation_mode",
                                        ]
                                    },
                                    {
                                        "$eq": [
                                            "$$a_start_date_time",
                                            "$start_date_time",
                                        ]
                                    },
                                    {"$eq": ["$$a_end_date_time", "$end_date_time"]},
                                ]
                            }
                        }
                    }
                ],
                "as": "duplicates"
            }
        }
    ]
)

for row in rows:
    print(rows)

KeyboardInterrupt: 

### Task 7
a) Find the number of users that have started an activity in one day and ended the activity the next day.

b) List the transportation mode, user id and duration for these activities.

Assumption: As the question explicitly mentions transportation mode we do not count the rows in the Activity table with transportation mode equals NULL as relevant to this question. Thus, there are less rows in this sub task. However, if it is the case to include these NULL rows then we only need to remove the 'AND transportation_mode IS NOT NULL' part.

### Task 8

Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)

### Task 9
Find the top 15 users who have gained the most altitude meters.

### Task 10
Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 11
Find all users who have invalid activities, and the number of invalid activities per user.

### Task 12
Find all the users who have registered transportation_mode and their most used transportation_mode

In [8]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$group": {
            "_id": {
                "user_id": "$_id.user_id",
            },
            "max": {
                "$max": "$count"
            }, 

        }},
        {"$sort": {"_id.user_id": 1}},
    ]
)

for row in rows:
    pprint(row)

{'_id': {'user_id': '010'}, 'max': 3}
{'_id': {'user_id': '020'}, 'max': 80}
{'_id': {'user_id': '021'}, 'max': 1}
{'_id': {'user_id': '052'}, 'max': 1}
{'_id': {'user_id': '056'}, 'max': 15}
{'_id': {'user_id': '058'}, 'max': 2}
{'_id': {'user_id': '060'}, 'max': 1}
{'_id': {'user_id': '062'}, 'max': 173}
{'_id': {'user_id': '064'}, 'max': 1}
{'_id': {'user_id': '065'}, 'max': 10}
{'_id': {'user_id': '067'}, 'max': 1}
{'_id': {'user_id': '069'}, 'max': 1}
{'_id': {'user_id': '073'}, 'max': 52}
{'_id': {'user_id': '075'}, 'max': 1}
{'_id': {'user_id': '076'}, 'max': 3}
{'_id': {'user_id': '078'}, 'max': 37}
{'_id': {'user_id': '080'}, 'max': 1}
{'_id': {'user_id': '081'}, 'max': 4}
{'_id': {'user_id': '082'}, 'max': 2}
{'_id': {'user_id': '084'}, 'max': 9}
{'_id': {'user_id': '085'}, 'max': 18}
{'_id': {'user_id': '086'}, 'max': 2}
{'_id': {'user_id': '087'}, 'max': 5}
{'_id': {'user_id': '089'}, 'max': 7}
{'_id': {'user_id': '091'}, 'max': 2}
{'_id': {'user_id': '092'}, 'max': 1}
{'_i